## Google places API

En este codigo se obtienen todas la informacion relevante (ubicacion, puntuacion, reseñas, etc) de los restaurantes de NY para su posterior analisis. 

Se emplea una tecnica de division hexagonal y paginacion para extraer todos los posibles resultados. Finalmente se eliminan los duplicados generados por solapamiento en la busqueda

### Librerias necesarias
h3 pandas requests tqdm python-dotenv

In [2]:
import h3
import requests
import pandas as pd
from tqdm import tqdm
import time
from dotenv import load_dotenv
import os

In [3]:
# Cargar variables de entorno
load_dotenv()
API_KEY = os.getenv('GOOGLE_API_KEY')


In [4]:
# Área aproximada de NYC en coordenadas (latitud, longitud)
NYC_BOUNDS = {
    'north': 40.9176,
    'south': 40.4774,
    'west': -74.2591,
    'east': -73.7004
}

# Resolución H3 (puedes probar 8 o 9; 9 = más detalle, más hexágonos)
H3_RESOLUTION = 9

In [5]:
# Función para generar hexágonos en NYC
def generate_hexagons(bounds, resolution):
    lat_min, lat_max = bounds['south'], bounds['north']
    lon_min, lon_max = bounds['west'], bounds['east']
    step = 0.01  # para escanear puntos dentro del área
    hexes = set()

    lat = lat_min
    while lat < lat_max:
        lon = lon_min
        while lon < lon_max:
            hex_id = h3.geo_to_h3(lat, lon, resolution)
            hexes.add(hex_id)
            lon += step
        lat += step
    return list(hexes)

In [6]:
# Nearby Search API call
def search_places_nearby(lat, lng, radius=1000):
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': f'{lat},{lng}',
        'radius': radius,
        'type': 'restaurant',
        'key': API_KEY
    }

    results = []
    while True:
        response = requests.get(url, params=params)
        data = response.json()
        results.extend(data.get('results', []))

        next_page_token = data.get('next_page_token')
        if not next_page_token:
            break
        time.sleep(2)
        params = {'pagetoken': next_page_token, 'key': API_KEY}
    return results